In [1]:
import pandas as pd
import numpy as np
import sklearn

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch

from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit,train_epoch
import numpy as np


from datasets import TripletSampling, BatchPairSampling
from networks import EmbeddingNet, SiameseNet
from losses import TripletLoss, HardNegativesBatchLoss, EasyHardNegativesBatchLoss

cuda = torch.cuda.is_available()

### Test loss Find All hardest case in a batch

In [46]:
features = torch.rand(4,5)
labels = torch.tensor( np.array([0]*2 + [1]*2) )
import torch.nn.functional as F

In [55]:
# helper
def Mat(Lvec):
    N = Lvec.size(0)
    Mask = Lvec.repeat(N,1)
    Same = (Mask==Mask.t())
    return Same.clone().fill_diagonal_(0), ~Same#same diff

def fun_dissim(Mat_A, Mat_B):#N by F
    
    
    return torch.cdist(Mat_A, Mat_B, p = 2)
    

In [94]:
Lvec = labels
fvec = features
margin = 1

# number of samples in a batch
N = Lvec.size(0)

# feature normalization
fvec_norm = F.normalize(fvec, p = 2, dim = 1)

# Same/Diff label Matting in Similarity Matrix
Same, Diff = Mat(Lvec.view(-1))

# dissimilarity Matrix
dissim = fun_dissim(fvec_norm, fvec_norm)

In [95]:
dissim

tensor([[0.0000, 1.0397, 0.5372, 0.8163],
        [1.0397, 0.0000, 0.6890, 0.2997],
        [0.5372, 0.6890, 0.0000, 0.4505],
        [0.8163, 0.2997, 0.4505, 0.0000]])

In [96]:
Same, Diff

(tensor([[False,  True, False, False],
         [ True, False, False, False],
         [False, False, False,  True],
         [False, False,  True, False]]),
 tensor([[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]]))

In [97]:
# finding the max per row
D_detach_P = dissim.clone().detach()

D_detach_P[Diff] = -1 # find on same label

V_pos, I_pos = D_detach_P.max(1) # find max per row, v_pos is value, I_pos index

# # extracting pos score
Pos = dissim[torch.arange(0,N), I_pos] # grab the max of each row
Pos = Pos.clone().detach().cpu()

In [98]:
Pos

tensor([1.0397, 1.0397, 0.4505, 0.4505])

In [99]:
# finding the max per row
D_detach_P = dissim.clone().detach()

D_detach_P[Same] = 10000 # set same label = 1000, don't want these
zero_mask = D_detach_P == 0
D_detach_P[zero_mask ] = 1000 # want the min but not 0

V_neg, I_neg = D_detach_P.min(1) # find min per row, v_ is value, I_ index

# # extracting pos score
Neg = dissim[torch.arange(0,N), I_neg] # grab the max of each row
Neg = Neg.clone().detach().cpu()

In [102]:
HardTripletMask = Neg < Pos
HardTriplet_num = sum(HardTripletMask).item()
losses = F.relu( Pos[HardTripletMask] - Neg[HardTripletMask] + margin).sum()
losses.

tensor(4.3934)

### Investigate Nan Loss

In [4]:
import torch
import torch.nn.functional as F

F.log_softmax(Triplet_val[:]/0.1, dim=1)[:,0].mean()

NameError: name 'Triplet_val' is not defined

In [26]:
normal_setup_a = torch.tensor([100])
normal_setup_b = torch.tensor([1])
stack_normal = torch.stack( [normal_setup_a, normal_setup_b], 1)

In [27]:
# F.log_softmax(stack_normal[:]/0.1, dim=1)[:,0].mean()
F.log_softmax(stack_normal[:]/0.1, dim=1)[:,0].shape

torch.Size([1])

In [31]:
-F.log_softmax(stack_normal[:]/0.1, dim=1)
F.softmax(stack_normal[:]/0.1, dim=1)

tensor([[1., 0.]])

In [44]:
a = torch.Tensor([0, float('inf')])
A = a.repeat(3,1)




In [53]:
F.log_softmax(torch.Tensor([]), dim = 1)  # should give [0.0, 1.0]

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)